In [2]:
import sys
import os
import pandas as pd

sys.path.insert(0, os.path.abspath('../..'))

from src.config import REPORTS_DIR

Util script to report the results of Predictive and Hybrid architecture into latex tables.

# Predictive Architecture

In [3]:
def extract_avg_metrics(folder_name, currency_filename):
    file_path = f'/{REPORTS_DIR}/{folder_name}/{currency_filename}'
    df = pd.read_csv(file_path)
    
    # Calculate average metrics
    avg_metrics = df.mean(numeric_only=True)
    
    return avg_metrics.to_dict()

def generate_latex_table_with_avg(metrics_dict, currency_files):
    header = "\\begin{tabular}{llrrrrrr}\n\\hline\n"
    header += "Currency Pair & Model & Log Loss & Accuracy & AUC & Returns & SD & SR \\\\ \\hline\n"
    
    rows = []
    avg_results = {metric: {folder: [] for folder in folder_names} for metric in ['Log Loss', 'Accuracy', 'AUC', 'Returns', 'SD', 'SR']}
    
    # Find top values for each metric and currency
    top_values = {currency_file: {metric: None for metric in avg_results.keys()} for currency_file in currency_files}

    for currency_file in currency_files:
        for metric in avg_results.keys():
            values = [metrics[currency_file][metric] for folder, metrics in metrics_dict.items() if currency_file in metrics and metric in metrics[currency_file]]
            if values:  # Check if values list is not empty
                top_value = min(values) if metric in ['Log Loss', 'SD'] else max(values)
                top_values[currency_file][metric] = top_value
                

    for currency_file in currency_files:
        currency = currency_file.replace(".csv", "")
        first_row = True
        for folder, metrics in metrics_dict.items():
            if currency_file in metrics:
                for metric in avg_results:
                    if metric in metrics[currency_file]:  # Check if the metric exists for the folder and currency file
                        avg_results[metric][folder].append(metrics[currency_file][metric])

                row = ""
                if first_row:
                    row += f"{currency} & "
                    first_row = False
                else:
                    row += " & "
                row += f"{folder} "
                for metric in ['Log Loss', 'Accuracy', 'AUC', 'Returns', 'SD', 'SR']:
                    if metric in metrics[currency_file]:  # Check if the metric exists for the folder and currency file
                        value = metrics[currency_file][metric]
                        if round(value, 4) == round(top_values[currency_file][metric], 4):
                            row += f"& \\textbf{{{value:.4f}}} "
                        else:
                            row += f"& {value:.4f} "
                    else:
                        row += "& N/A "  # Add placeholder for missing metric
                row += "\\\\ \n"
                rows.append(row)
        rows.append("\\hline\n")

    
    # Find top values for AVG
    top_avg_values = {metric: None for metric in avg_results.keys()}
    for metric in avg_results.keys():
        avg_values = [sum(avg_results[metric][folder])/len(avg_results[metric][folder]) if avg_results[metric][folder] else 0 for folder in folder_names]
        top_avg_value = min(avg_values) if metric in ['Log Loss', 'SD'] else max(avg_values)
        top_avg_values[metric] = top_avg_value

    # Add average results
    first_row = True
    for folder in folder_names:
        if first_row:
            row = f"AVG & {folder} "
            first_row = False
        else:
            row = f" & {folder} "
        for metric in ['Log Loss', 'Accuracy', 'AUC', 'Returns', 'SD', 'SR']:
            if avg_results[metric][folder]:  # Check if the list is not empty
                avg_value = sum(avg_results[metric][folder])/len(avg_results[metric][folder])
                if round(avg_value, 4) == round(top_avg_values[metric], 4):
                    row += f"& \\textbf{{{avg_value:.4f}}} "
                else:
                    row += f"& {avg_value:.4f} "
            else:
                row += "& N/A "  # Add placeholder for missing metric
        row += "\\\\ \n"
        rows.append(row)

    
    table = header + "".join(rows) + "\\end{tabular}"
    
    return table


folder_names = ["naive", "LR", "RF", "ANN", "RNN", "LSTM", "GRU"]
currency_files = ["EURUSD.csv", "GBPUSD.csv", "USDCHF.csv", "USDJPY.csv"]

results = {}

for folder in folder_names:
    results[folder] = {}
    for currency_file in currency_files:
        try:
            results[folder][currency_file] = extract_avg_metrics(folder, currency_file)
        except:
            continue

latex_table_with_avg = generate_latex_table_with_avg(results, currency_files)
print(latex_table_with_avg)


\begin{tabular}{llrrrrrr}
\hline
Currency Pair & Model & Log Loss & Accuracy & AUC & Returns & SD & SR \\ \hline
EURUSD & naive & N/A & 0.4931 & 0.4931 & -0.1297 & \textbf{0.0557} & -0.2131 \\ 
 & LR & 1.8079 & 0.4827 & 0.4896 & -0.0498 & 0.0709 & -0.6892 \\ 
 & RF & 0.6978 & 0.4793 & 0.4863 & -0.0312 & 0.0708 & -0.4142 \\ 
 & ANN & 0.7360 & 0.5000 & 0.4925 & 0.0058 & 0.0710 & 0.0590 \\ 
 & RNN & 0.7147 & 0.5187 & 0.5130 & \textbf{0.0465} & 0.0708 & \textbf{0.4615} \\ 
 & LSTM & \textbf{0.6911} & \textbf{0.5333} & \textbf{0.5457} & 0.0300 & 0.0709 & 0.4152 \\ 
 & GRU & \textbf{0.6911} & 0.5087 & 0.5424 & -0.0143 & 0.0709 & -0.0465 \\ 
\hline
GBPUSD & naive & N/A & 0.5100 & 0.5098 & \textbf{0.0981} & \textbf{0.0671} & 0.1687 \\ 
 & LR & 1.8000 & 0.5107 & 0.5145 & 0.0064 & 0.0952 & 0.1935 \\ 
 & RF & 0.6963 & 0.5220 & 0.5292 & 0.0615 & 0.0951 & 0.5691 \\ 
 & ANN & 0.7304 & 0.5207 & 0.5375 & 0.0402 & 0.0951 & 0.4165 \\ 
 & RNN & 0.7078 & \textbf{0.5280} & \textbf{0.5408} & 0.0943 & 0.0950

# Hybrid Architecture

In [4]:
# V1
# Define the order of folders
ordered_folders = ["LR_clustering", "RF_clustering", "ANN_clustering", "RNN_clustering", "LSTM_clustering", "GRU_clustering"]

# Define the list of currency pairs based on the file names
currency_files_ma5 = ["EURUSD_MA5.csv", "GBPUSD_MA5.csv", "USDCHF_MA5.csv", "USDJPY_MA5.csv"]
currency_files_ma10 = ["EURUSD_MA10.csv", "GBPUSD_MA10.csv", "USDCHF_MA10.csv", "USDJPY_MA10.csv"]

currency_pairs = [filename.split("_")[0] for filename in currency_files_ma5]

def extract_avg_metrics_clustering_updated(folder_path, currency_file_ma5, currency_file_ma10):
    ma5_data = pd.read_csv(os.path.join(folder_path, currency_file_ma5))
    ma10_data = pd.read_csv(os.path.join(folder_path, currency_file_ma10))
    
    avg_metrics_ma5 = [round(ma5_data[col].mean(), 4) for col in ma5_data.columns[1:]]
    avg_metrics_ma10 = [round(ma10_data[col].mean(), 4) for col in ma10_data.columns[1:]]
    
    return avg_metrics_ma5, avg_metrics_ma10

def generate_latex_table_with_avg_clustering_ordered_final_updated_v8(results_clustering, currency_files_ma5, currency_files_ma10):
    header = "\\begin{tabular}{@{} l l r r r r r r r r r r r r @{}}\n"
    header += "\\toprule\n"
    header += "& & \\multicolumn{6}{c}{MA5} & \\multicolumn{6}{c}{MA10} \\\\\n"
    header += "\\cmidrule(lr){3-8}\\cmidrule(lr){9-14}\n"
    header += "Currency Pair & Model & Log Loss & Accuracy & AUC & Returns & SD & SR & Log Loss & Accuracy & AUC & Returns & SD & SR \\\\\n"
    header += "\\midrule\n"


    body = ""
    for currency_pair, currency_file_ma5, currency_file_ma10 in zip(currency_pairs, currency_files_ma5, currency_files_ma10):
        for folder in ordered_folders:
            model_name = folder.replace("_clustering", "") # Removing "_clustering" postfix
            metric_ma5 = ["{:.4f}".format(value) for value in results_clustering[folder][currency_file_ma5]['ma5']]
            metric_ma10 = ["{:.4f}".format(value) for value in results_clustering[folder][currency_file_ma10]['ma10']]
            body += f"{currency_pair} & {model_name} & {' & '.join(metric_ma5)} & {' & '.join(metric_ma10)} \\\\ \n"
            currency_pair = ""  # Clearing the currency name for the subsequent rows
        body += "\\midrule\n"
    
    # Remove the last "\midrule\n" to avoid consecutive lines
    body = body.rstrip("\\midrule\n")
    
    # Adding AVG subsection
    body += "\\midrule\n"
    avg_label = "AVG"
    for folder in ordered_folders:
        model_name = folder.replace("_clustering", "")
        avg_metrics_ma5 = ["{:.4f}".format(round(sum([results_clustering[folder][currency_file].get('ma5', [0]*6)[i] for currency_file in currency_files_ma5]) / len(currency_files_ma5), 4)) for i in range(6)]
        avg_metrics_ma10 = ["{:.4f}".format(round(sum([results_clustering[folder][currency_file].get('ma10', [0]*6)[i] for currency_file in currency_files_ma10]) / len(currency_files_ma10), 4)) for i in range(6)]
        body += f"{avg_label} & {model_name} & {' & '.join(avg_metrics_ma5)} & {' & '.join(avg_metrics_ma10)} \\\\ \n"
        avg_label = ""  # Clearing the AVG label for the subsequent rows
    
    footer = "\\bottomrule\n"
    footer += "\\end{tabular}"
    
    return header + body + footer


results_clustering = {}
for folder in ordered_folders:
    folder_path = os.path.join(REPORTS_DIR, folder)
    results_clustering[folder] = {}
    for currency_file_ma5, currency_file_ma10 in zip(currency_files_ma5, currency_files_ma10):
        results_clustering[folder][currency_file_ma5] = {'ma5': {}}
        results_clustering[folder][currency_file_ma10] = {'ma10': {}}
        results_clustering[folder][currency_file_ma5]['ma5'], results_clustering[folder][currency_file_ma10]['ma10'] = \
            extract_avg_metrics_clustering_updated(folder_path, currency_file_ma5, currency_file_ma10)

latex_table_with_avg_clustering = generate_latex_table_with_avg_clustering_ordered_final_updated_v8(results_clustering, currency_files_ma5, currency_files_ma10)
print(latex_table_with_avg_clustering)


\begin{tabular}{@{} l l r r r r r r r r r r r r @{}}
\toprule
& & \multicolumn{6}{c}{MA5} & \multicolumn{6}{c}{MA10} \\
\cmidrule(lr){3-8}\cmidrule(lr){9-14}
Currency Pair & Model & Log Loss & Accuracy & AUC & Returns & SD & SR & Log Loss & Accuracy & AUC & Returns & SD & SR \\
\midrule
EURUSD & LR & 1.7728 & 0.5147 & 0.5177 & -0.0648 & 0.0706 & -1.0996 & 1.8621 & 0.5060 & 0.5165 & -0.0633 & 0.0708 & -0.9374 \\ 
 & RF & 0.6982 & 0.4967 & 0.4887 & -0.0371 & 0.0708 & -0.6850 & 0.6960 & 0.5007 & 0.5134 & -0.0645 & 0.0707 & -0.9312 \\ 
 & ANN & 0.7303 & 0.5040 & 0.4914 & -0.0450 & 0.0709 & -0.6433 & 0.7305 & 0.5013 & 0.4962 & -0.0404 & 0.0707 & -0.6655 \\ 
 & RNN & 0.7118 & 0.5000 & 0.5219 & -0.0641 & 0.0699 & -1.5951 & 0.7141 & 0.5067 & 0.5124 & -0.1168 & 0.0687 & -2.8607 \\ 
 & LSTM & 0.6873 & 0.5167 & 0.5589 & -0.1962 & 0.0695 & -3.2214 & 0.6865 & 0.5327 & 0.5674 & -0.1742 & 0.0699 & -2.7266 \\ 
 & GRU & 0.6870 & 0.5180 & 0.5637 & -0.2659 & 0.0680 & -4.7537 & 0.6873 & 0.5240 & 0.5669 & 

In [5]:
import os
import pandas as pd

# Define the order of folders
ordered_folders = ["LR_clustering", "RF_clustering", "ANN_clustering", "RNN_clustering", "LSTM_clustering", "GRU_clustering"]

# Define the list of currency pairs based on the file names
currency_files_ma5 = ["EURUSD_MA5.csv", "GBPUSD_MA5.csv", "USDCHF_MA5.csv", "USDJPY_MA5.csv"]
currency_files_ma10 = ["EURUSD_MA10.csv", "GBPUSD_MA10.csv", "USDCHF_MA10.csv", "USDJPY_MA10.csv"]

currency_pairs = [filename.split("_")[0] for filename in currency_files_ma5]

def extract_avg_metrics_clustering_updated(folder_path, currency_file_ma5, currency_file_ma10):
    ma5_data = pd.read_csv(os.path.join(folder_path, currency_file_ma5))
    ma10_data = pd.read_csv(os.path.join(folder_path, currency_file_ma10))
    
    avg_metrics_ma5 = [round(ma5_data[col].mean(), 4) for col in ma5_data.columns[1:]]
    avg_metrics_ma10 = [round(ma10_data[col].mean(), 4) for col in ma10_data.columns[1:]]
    
    return avg_metrics_ma5, avg_metrics_ma10

def generate_latex_table_with_avg_clustering_ordered_final_updated_v8(results_clustering, currency_files_ma5, currency_files_ma10):
    header = "\\begin{tabular}{@{} l l r r r r r r r r r r r r @{}}\n"
    header += "\\toprule\n"
    header += "& & \\multicolumn{6}{c}{MA5} & \\multicolumn{6}{c}{MA10} \\\\\n"
    header += "\\cmidrule(lr){3-8}\\cmidrule(lr){9-14}\n"
    header += "Currency Pair & Model & Log Loss & Accuracy & AUC & Returns & SD & SR & Log Loss & Accuracy & AUC & Returns & SD & SR \\\\\n"
    header += "\\midrule\n"

    # Find top values for each metric and currency for both MA5 and MA10
    top_values_ma5 = {currency_file: [None]*6 for currency_file in currency_files_ma5}
    top_values_ma10 = {currency_file: [None]*6 for currency_file in currency_files_ma10}
    
    for currency_file_ma5, currency_file_ma10 in zip(currency_files_ma5, currency_files_ma10):
        for i in range(6):  # 6 metrics
            values_ma5 = [results_clustering[folder][currency_file_ma5]['ma5'][i] for folder in ordered_folders]
            values_ma10 = [results_clustering[folder][currency_file_ma10]['ma10'][i] for folder in ordered_folders]
            
            top_values_ma5[currency_file_ma5][i] = min(values_ma5) if i == 0 else max(values_ma5)  # Assuming Log Loss is the first metric
            top_values_ma10[currency_file_ma10][i] = min(values_ma10) if i == 0 else max(values_ma10)

    body = ""
    for currency_pair, currency_file_ma5, currency_file_ma10 in zip(currency_pairs, currency_files_ma5, currency_files_ma10):
        for folder in ordered_folders:
            model_name = folder.replace("_clustering", "")
            metric_ma5 = []
            metric_ma10 = []
            for i in range(6):
                value_ma5 = results_clustering[folder][currency_file_ma5]['ma5'][i]
                value_ma10 = results_clustering[folder][currency_file_ma10]['ma10'][i]
                
                if round(value_ma5, 4) == round(top_values_ma5[currency_file_ma5][i], 4):
                    metric_ma5.append(f"\\textbf{{{value_ma5:.4f}}}")
                else:
                    metric_ma5.append(f"{value_ma5:.4f}")
                
                if round(value_ma10, 4) == round(top_values_ma10[currency_file_ma10][i], 4):
                    metric_ma10.append(f"\\textbf{{{value_ma10:.4f}}}")
                else:
                    metric_ma10.append(f"{value_ma10:.4f}")

            body += f"{currency_pair} & {model_name} & {' & '.join(metric_ma5)} & {' & '.join(metric_ma10)} \\\\ \n"
            currency_pair = ""

    # Adding AVG subsection
    body += "\\midrule\n"
    avg_label = "AVG"
    top_avg_values_ma5 = [None]*6
    top_avg_values_ma10 = [None]*6
    for i in range(6):
        avg_values_ma5 = [round(sum([results_clustering[folder][currency_file].get('ma5', [0]*6)[i] for currency_file in currency_files_ma5]) / len(currency_files_ma5), 4) for folder in ordered_folders]
        avg_values_ma10 = [round(sum([results_clustering[folder][currency_file].get('ma10', [0]*6)[i] for currency_file in currency_files_ma10]) / len(currency_files_ma10), 4) for folder in ordered_folders]
        top_avg_values_ma5[i] = min(avg_values_ma5) if i == 0 else max(avg_values_ma5)
        top_avg_values_ma10[i] = min(avg_values_ma10) if i == 0 else max(avg_values_ma10)

    for folder in ordered_folders:
        model_name = folder.replace("_clustering", "")
        avg_metrics_ma5 = []
        avg_metrics_ma10 = []
        for i in range(6):
            avg_value_ma5 = round(sum([results_clustering[folder][currency_file].get('ma5', [0]*6)[i] for currency_file in currency_files_ma5]) / len(currency_files_ma5), 4)
            avg_value_ma10 = round(sum([results_clustering[folder][currency_file].get('ma10', [0]*6)[i] for currency_file in currency_files_ma10]) / len(currency_files_ma10), 4)
            
            if round(avg_value_ma5, 4) == round(top_avg_values_ma5[i], 4):
                avg_metrics_ma5.append(f"\\textbf{{{avg_value_ma5:.4f}}}")
            else:
                avg_metrics_ma5.append(f"{avg_value_ma5:.4f}")
            
            if round(avg_value_ma10, 4) == round(top_avg_values_ma10[i], 4):
                avg_metrics_ma10.append(f"\\textbf{{{avg_value_ma10:.4f}}}")
            else:
                avg_metrics_ma10.append(f"{avg_value_ma10:.4f}")

        body += f"{avg_label} & {model_name} & {' & '.join(avg_metrics_ma5)} & {' & '.join(avg_metrics_ma10)} \\\\ \n"
        avg_label = ""  # Clearing the AVG label for the subsequent rows
    
    footer = "\\bottomrule\n"
    footer += "\\end{tabular}"
    
    return header + body + footer

results_clustering = {}
for folder in ordered_folders:
    folder_path = os.path.join(REPORTS_DIR, folder)
    results_clustering[folder] = {}
    for currency_file_ma5, currency_file_ma10 in zip(currency_files_ma5, currency_files_ma10):
        results_clustering[folder][currency_file_ma5] = {'ma5': {}}
        results_clustering[folder][currency_file_ma10] = {'ma10': {}}
        results_clustering[folder][currency_file_ma5]['ma5'], results_clustering[folder][currency_file_ma10]['ma10'] = \
            extract_avg_metrics_clustering_updated(folder_path, currency_file_ma5, currency_file_ma10)

latex_table_with_avg_clustering = generate_latex_table_with_avg_clustering_ordered_final_updated_v8(results_clustering, currency_files_ma5, currency_files_ma10)
print(latex_table_with_avg_clustering)


\begin{tabular}{@{} l l r r r r r r r r r r r r @{}}
\toprule
& & \multicolumn{6}{c}{MA5} & \multicolumn{6}{c}{MA10} \\
\cmidrule(lr){3-8}\cmidrule(lr){9-14}
Currency Pair & Model & Log Loss & Accuracy & AUC & Returns & SD & SR & Log Loss & Accuracy & AUC & Returns & SD & SR \\
\midrule
EURUSD & LR & 1.7728 & 0.5147 & 0.5177 & -0.0648 & 0.0706 & -1.0996 & 1.8621 & 0.5060 & 0.5165 & -0.0633 & \textbf{0.0708} & -0.9374 \\ 
 & RF & 0.6982 & 0.4967 & 0.4887 & \textbf{-0.0371} & 0.0708 & -0.6850 & 0.6960 & 0.5007 & 0.5134 & -0.0645 & 0.0707 & -0.9312 \\ 
 & ANN & 0.7303 & 0.5040 & 0.4914 & -0.0450 & \textbf{0.0709} & \textbf{-0.6433} & 0.7305 & 0.5013 & 0.4962 & \textbf{-0.0404} & 0.0707 & \textbf{-0.6655} \\ 
 & RNN & 0.7118 & 0.5000 & 0.5219 & -0.0641 & 0.0699 & -1.5951 & 0.7141 & 0.5067 & 0.5124 & -0.1168 & 0.0687 & -2.8607 \\ 
 & LSTM & 0.6873 & 0.5167 & 0.5589 & -0.1962 & 0.0695 & -3.2214 & \textbf{0.6865} & \textbf{0.5327} & \textbf{0.5674} & -0.1742 & 0.0699 & -2.7266 \\ 
 & GRU & \t

# Expanded Results

In [6]:
def generate_latex_table_final_fixed_v34(model: str, currency_pair: str, filepath: str) -> str:
    data = pd.read_csv(filepath)
    
    # Drop the unnamed index column if present
    data = data.drop(columns=['Unnamed: 0'], errors='ignore')
    
    # Adjust the first column to have "Period 1", "Period 2", ... and "AVG" for the last row
    periods = ["Period " + str(i+1) for i in range(len(data)-1)]
    periods.append("AVG")  # Add "AVG" for the last row
    data.insert(0, "Period", periods)
    
    # Round to 4 decimal places and format as string
    for col in data.columns[1:]:
        data[col] = data[col].apply(lambda x: f"{x:.4f}")
    
    # Highlight the best scores (excluding the last row, which is the AVG)
    data_highlighted = data.iloc[:-1].copy()
    for col in data_highlighted.columns[1:]:
        is_higher_better = col == "Returns" or col == "Accuracy" or col == "AUC" or col == "SR"
        if is_higher_better:
            best_score_idx = data_highlighted[col].astype(float).idxmax()
        else:
            best_score_idx = data_highlighted[col].astype(float).idxmin()
        data_highlighted.at[best_score_idx, col] = "\\textbf{" + data_highlighted.at[best_score_idx, col] + "}"
    data.update(data_highlighted)
    
    # Determine the number of columns in the data
    num_columns = len(data.columns)
    column_format = 'l' + 'c' * (num_columns - 1)
    
    # Convert DataFrame to LaTeX format with the correct style
    header = "\\toprule \nPeriod & " + " & ".join(data.columns[1:]) + " \\\\ \n\\midrule"
    footer = "\\bottomrule"
    
    # Insert a midrule before the AVG row
    latex_table = data.to_latex(index=False, column_format=column_format, header=False, escape=False)
    #latex_table = data.style.to_latex(index=False, column_format=column_format, header=False, escape=False)

    
    avg_row_idx = latex_table.rfind("AVG")
    latex_table = latex_table[:avg_row_idx] + "\\midrule\n" + latex_table[avg_row_idx:]
    
    latex_table = latex_table.replace("\\toprule", header).replace("\\bottomrule", footer)
    
    #return "\\begin{table}[h]\n\\centering" + latex_table + f"\n\\caption{{Results for {model} on {currency_pair}}}\n\\label{{tab:{model}_{currency_pair}}}\n\\end{{table}}"
    return latex_table

# List of models and currency pairs
#models = ["ANN", "GRU", "LR", "LSTM", "RF", "RNN"]
models = ["LR", "RF", "ANN", "RNN", "LSTM", "GRU"]

currency_pairs = ["EURUSD", "GBPUSD", "USDCHF", "USDJPY"]

latex_tables = {}  # Dictionary to store LaTeX tables for each model and currency pair

# Loop through each model
for model in models:
    model_path = os.path.join(REPORTS_DIR, model)

    # Loop through each currency pair for the model
    for currency in currency_pairs:
        file_path = os.path.join(model_path, f"{currency}.csv")

        # Check if the file exists
        if os.path.exists(file_path):
            try:
                # Generate the LaTeX table and store it in the dictionary
                latex_tables[(model, currency)] = generate_latex_table_final_fixed_v34(model, currency, file_path)
            except Exception as e:
                print(f"Error processing {model} - {currency}: {str(e)}")
                # Print the problematic data for debugging
                data = pd.read_csv(file_path)
                print(data)

#print(latex_tables.get(('ANN', 'EURUSD')))

#print(latex_tables)

"""
for k, v in latex_tables.items():
    print(v)
"""

latex_template = r"""
\begin{figure}
    \centering
    \vspace{5pt}
    \subcaptionbox{EURUSD}{
        \resizebox{0.80\textwidth}{!}{%
            %SUBITEM1
        }
    }
    \vspace{5pt}
    \subcaptionbox{GBPUSD}{
        \resizebox{0.80\textwidth}{!}{%
            %SUBITEM2
        }
    }
    \vspace{5pt}
    \subcaptionbox{USDCHF}{
        \resizebox{0.80\textwidth}{!}{%
            %SUBITEM3
        }
    }
    \vspace{5pt}
    \subcaptionbox{USDJPY}{
        \resizebox{0.80\textwidth}{!}{%
            %SUBITEM4
        }
    }
    \caption[]{
        Expanded results for %MODEL.
    }
\end{figure}
"""

for model in models:
    
    cur_template = latex_template.replace("%SUBITEM1", latex_tables.get((model, "EURUSD")))
    cur_template = cur_template.replace("%SUBITEM2", latex_tables.get((model, "GBPUSD")))
    cur_template = cur_template.replace("%SUBITEM3", latex_tables.get((model, "USDCHF")))
    cur_template = cur_template.replace("%SUBITEM4", latex_tables.get((model, "USDJPY")))
    cur_template = cur_template.replace("%MODEL", model)
    
    print(cur_template)
    
    #for pair in currency_pairs:
    #print(latex_tables.get((model, pair)))





\begin{figure}
    \centering
    \vspace{5pt}
    \subcaptionbox{EURUSD}{
        \resizebox{0.80\textwidth}{!}{%
            \begin{tabular}{lcccccc}
\toprule 
Period & Log Loss & Accuracy & AUC & Returns & SD & SR \\ 
\midrule
Period 1 &          2.0748 &          0.4720 &          0.4800 &          -0.0752 &          0.0875 &          -0.8567 \\
Period 2 &          2.1132 &          0.4760 &          0.4811 &          -0.1006 &          0.0875 &          -1.1775 \\
Period 3 &          1.4570 & \textbf{0.5080} & \textbf{0.5241} & \textbf{-0.0129} &          0.0725 & \textbf{-0.1446} \\
Period 4 &          1.7250 &          0.4560 &          0.4696 &          -0.0774 & \textbf{0.0541} &          -1.4729 \\
Period 5 & \textbf{1.2141} &          0.4960 &          0.5236 &          -0.0139 &          0.0661 &          -0.1803 \\
Period 6 &          2.2630 &          0.4880 &          0.4593 &          -0.0190 &          0.0580 &          -0.3035 \\
     \midrule
AVG &          1.8079 &

/var/folders/ms/z80j1cn1263cmvfhbq8191cm0000gn/T/ipykernel_40271/2157620280.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = data.to_latex(index=False, column_format=column_format, header=False, escape=False)
/var/folders/ms/z80j1cn1263cmvfhbq8191cm0000gn/T/ipykernel_40271/2157620280.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = data.to_latex(index=False, column_format=column_format, header=False, escape=False)
/var/folders/ms/z80j1cn1

In [7]:
models = ["LR_clustering", "RF_clustering", "ANN_clustering", "RNN_clustering", "LSTM_clustering", "GRU_clustering"]

currency_pairs = ["EURUSD", "GBPUSD", "USDCHF", "USDJPY"]

latex_tables = {}  # Dictionary to store LaTeX tables for each model and currency pair

# Loop through each model
for model in models:
    model_path = os.path.join(REPORTS_DIR, model)

    # Loop through each currency pair for the model
    for currency in currency_pairs:
        file_path = os.path.join(model_path, f"{currency}_MA5.csv")

        # Check if the file exists
        if os.path.exists(file_path):
            try:
                # Generate the LaTeX table and store it in the dictionary
                latex_tables[(model, currency)] = generate_latex_table_final_fixed_v34(model, currency, file_path)
            except Exception as e:
                print(f"Error processing {model} - {currency}: {str(e)}")
                # Print the problematic data for debugging
                data = pd.read_csv(file_path)
                print(data)

 
for model in models:
    
    cur_template = latex_template.replace("%SUBITEM1", latex_tables.get((model, "EURUSD")))
    cur_template = cur_template.replace("%SUBITEM2", latex_tables.get((model, "GBPUSD")))
    cur_template = cur_template.replace("%SUBITEM3", latex_tables.get((model, "USDCHF")))
    cur_template = cur_template.replace("%SUBITEM4", latex_tables.get((model, "USDJPY")))
    cur_template = cur_template.replace("%MODEL", model).replace("_clustering", " with clustering MA5")
    
    print(cur_template)
    



\begin{figure}
    \centering
    \vspace{5pt}
    \subcaptionbox{EURUSD}{
        \resizebox{0.80\textwidth}{!}{%
            \begin{tabular}{lcccccc}
\toprule 
Period & Log Loss & Accuracy & AUC & Returns & SD & SR \\ 
\midrule
Period 1 &          1.8987 &          0.5160 &          0.5200 & \textbf{0.0552} &          0.0875 & \textbf{0.6627} \\
Period 2 &          1.7245 &          0.4800 &          0.4797 &         -0.0062 &          0.0877 &         -0.0280 \\
Period 3 & \textbf{1.3979} & \textbf{0.5720} & \textbf{0.5931} &         -0.1747 &          0.0715 &         -2.6705 \\
Period 4 &          1.7873 &          0.5120 &          0.5219 &         -0.0138 & \textbf{0.0543} &         -0.2296 \\
Period 5 &          1.5846 &          0.5200 &          0.5054 &         -0.2152 &          0.0643 &         -3.7625 \\
Period 6 &          2.2436 &          0.4880 &          0.4860 &         -0.0339 &          0.0580 &         -0.5698 \\
     \midrule
AVG &          1.7728 &          0.

/var/folders/ms/z80j1cn1263cmvfhbq8191cm0000gn/T/ipykernel_40271/2157620280.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = data.to_latex(index=False, column_format=column_format, header=False, escape=False)
/var/folders/ms/z80j1cn1263cmvfhbq8191cm0000gn/T/ipykernel_40271/2157620280.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = data.to_latex(index=False, column_format=column_format, header=False, escape=False)
/var/folders/ms/z80j1cn1

In [8]:
latex_tables = {}  # Dictionary to store LaTeX tables for each model and currency pair

# Loop through each model
for model in models:
    model_path = os.path.join(REPORTS_DIR, model)

    # Loop through each currency pair for the model
    for currency in currency_pairs:
        file_path = os.path.join(model_path, f"{currency}_MA10.csv")

        # Check if the file exists
        if os.path.exists(file_path):
            try:
                # Generate the LaTeX table and store it in the dictionary
                latex_tables[(model, currency)] = generate_latex_table_final_fixed_v34(model, currency, file_path)
            except Exception as e:
                print(f"Error processing {model} - {currency}: {str(e)}")
                # Print the problematic data for debugging
                data = pd.read_csv(file_path)
                print(data)

 
for model in models:
    
    cur_template = latex_template.replace("%SUBITEM1", latex_tables.get((model, "EURUSD")))
    cur_template = cur_template.replace("%SUBITEM2", latex_tables.get((model, "GBPUSD")))
    cur_template = cur_template.replace("%SUBITEM3", latex_tables.get((model, "USDCHF")))
    cur_template = cur_template.replace("%SUBITEM4", latex_tables.get((model, "USDJPY")))
    cur_template = cur_template.replace("%MODEL", model).replace("_clustering", " with clustering MA10")
    
    print(cur_template)
    



\begin{figure}
    \centering
    \vspace{5pt}
    \subcaptionbox{EURUSD}{
        \resizebox{0.80\textwidth}{!}{%
            \begin{tabular}{lcccccc}
\toprule 
Period & Log Loss & Accuracy & AUC & Returns & SD & SR \\ 
\midrule
Period 1 &          1.9763 &          0.4600 &          0.4754 &          0.0005 &          0.0876 &          0.0496 \\
Period 2 &          1.8873 &          0.5040 &          0.4901 &         -0.0682 &          0.0876 &         -0.7682 \\
Period 3 &          1.6190 & \textbf{0.5520} & \textbf{0.5687} &         -0.1749 &          0.0715 &         -2.6740 \\
Period 4 &          1.4171 &          0.5360 &          0.5530 &         -0.0345 & \textbf{0.0543} &         -0.6240 \\
Period 5 & \textbf{1.4018} &          0.5200 &          0.5305 &         -0.1175 &          0.0657 &         -1.8850 \\
Period 6 &          2.8711 &          0.4640 &          0.4815 & \textbf{0.0144} &          0.0580 & \textbf{0.2774} \\
     \midrule
AVG &          1.8621 &          0.

/var/folders/ms/z80j1cn1263cmvfhbq8191cm0000gn/T/ipykernel_40271/2157620280.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = data.to_latex(index=False, column_format=column_format, header=False, escape=False)
/var/folders/ms/z80j1cn1263cmvfhbq8191cm0000gn/T/ipykernel_40271/2157620280.py:36: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = data.to_latex(index=False, column_format=column_format, header=False, escape=False)
/var/folders/ms/z80j1cn1